In [1]:
!pip install tensorflow==2.12.0

In [2]:
import os
import re
import cv2
import time
import tarfile
import datetime
import warnings
import numpy as np
import pandas as pd
from PIL import Image
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow import concat
from tensorflow import repeat
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
warnings.filterwarnings('ignore')
from sklearn.utils import shuffle
from skimage.transform import resize
import nltk.translate.bleu_score as bleu
from tensorflow.keras.models import Model
from google.colab.patches import cv2_imshow
from tensorflow.keras.models import load_model
from tensorflow.keras.backend import expand_dims
from nltk.translate.bleu_score import sentence_bleu
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import concatenate, Concatenate
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.layers import Input, Softmax, RNN, Dense, Embedding, LSTM, Layer, Dropout, GRU

In [3]:
tf.__version__

'2.12.0'

In [4]:
import flask
from werkzeug.utils import secure_filename
from flask import Flask, redirect, url_for, request, render_template, jsonify

### Change dir to current working dir

In [5]:
os.chdir("/content/drive/MyDrive/deployment")

In [6]:
ls

data/  deployment.ipynb  models/  static/  templates/


### Loading CheXNet Model :

In [7]:
image_model = DenseNet121(weights='models/brucechou1983_CheXNet_Keras_0.3.0_weights.h5', classes = 14, input_shape=(256,256,3))
model = Model(image_model.input, image_model.layers[-2].output)

In [8]:
train = np.load('data/train.npy',allow_pickle=True)
test = np.load('data/test.npy',allow_pickle=True)
validation = np.load('data/validation.npy',allow_pickle=True)

columns = ["front X-Ray", "lateral X-Ray", "findings", "dec_ip", "dec_op", "image_features"]

train = pd.DataFrame(train, columns = columns)
test = pd.DataFrame(test, columns = columns)
validation = pd.DataFrame(validation, columns = columns)

#Reshaping the Image tensors for training
train_image_features = np.vstack(train.image_features).astype(np.float)
validation_image_features = np.vstack(validation.image_features).astype(np.float)

### Text Tokenization :

In [9]:

token = Tokenizer( filters='!"#$%&()*+,-/:;=?@[\\]^_`{|}~\t\n')
token.fit_on_texts(train['findings'])

token.word_index['<pad>'] = 0
token.index_word[0] = '<pad>'
vocab_size = len(token.word_index) + 1

#sequence in train and validation
train_inp_dec = token.texts_to_sequences(train.dec_ip)
train_op_dec = token.texts_to_sequences(train.dec_op)
val_inp_dec = token.texts_to_sequences(validation.dec_ip)
val_op_dec = token.texts_to_sequences(validation.dec_op)

#padding in the train and validation
max_len = 100
decoder_input = pad_sequences(train_inp_dec, maxlen=max_len, padding='post')
decoder_output =  pad_sequences(train_op_dec, maxlen=max_len, padding='post')
Validation_decoder_input = pad_sequences(val_inp_dec, maxlen=max_len, padding='post')
Validation_decoder_output = pad_sequences(val_op_dec, maxlen=max_len, padding='post')

word_idx = {}
idx_word = {}
for key, value in (token.word_index).items():
    word_idx[key] = value
    idx_word[value] = key

In [10]:
batch_size     = 50
Buffer_size    = 500

train_dataset = tf.data.Dataset.from_tensor_slices(((train_image_features, decoder_input), decoder_output))
train_dataset = train_dataset.shuffle(Buffer_size).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

validation_dataset = tf.data.Dataset.from_tensor_slices(((validation_image_features,Validation_decoder_input),Validation_decoder_output))
validation_dataset = validation_dataset.shuffle(Buffer_size).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


### Loading Model Architecture :

In [11]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''
    def __init__(self,lstm_units):
        super().__init__()

        self.lstm_units = lstm_units
        self.dense      = Dense(self.lstm_units, kernel_initializer="glorot_uniform", name = 'encoder_dense_layer')

    def initialize_states(self,batch_size):

        self.batch_size  = batch_size
        self.enc_h       = tf.zeros((self.batch_size, self.lstm_units))

        return self.enc_h

    def call(self,x):

        # x : image_data
        encoder_output = self.dense(x)

        return encoder_output


class Attention(tf.keras.layers.Layer):
    '''
    Class that calculates score based on the scoring_function using Bahdanu attention mechanism.
    '''
    def __init__(self,attention_units):
        super().__init__()

        self.attention_units = attention_units

        self.w1_Dense    =  tf.keras.layers.Dense(self.attention_units, kernel_initializer="glorot_uniform", name='Concat_w1_Dense')
        self.w2_Dense    =  tf.keras.layers.Dense(self.attention_units, kernel_initializer="glorot_uniform", name='Concat_w2_Dense')
        self.Concat_Dense=  tf.keras.layers.Dense(1, kernel_initializer="glorot_uniform", name = 'Concat_Dense_layer')

    def call(self,x):

        self.decoder_hidden_state, self.encoder_output = x
        self.decoder_hidden_state = tf.expand_dims(self.decoder_hidden_state,axis = 1)

        score = self.Concat_Dense(tf.nn.tanh(self.w1_Dense(self.decoder_hidden_state) + self.w2_Dense(self.encoder_output)))

        att_weights    = tf.nn.softmax(score, axis=1)
        context_vector = att_weights * self.encoder_output
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector,att_weights


class OneStepDecoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, lstm_units, attention_units):
        super().__init__()

        self.lstm_units     = lstm_units
        self.vocab_size     = vocab_size
        self.embedding_dim  = embedding_dim
        self.attention_units= attention_units

        self.dense       = Dense(self.vocab_size, kernel_initializer="glorot_uniform", name ='onestep_dense')
        self.attention   = Attention( self.attention_units)
        self.decoder_emb = Embedding(self.vocab_size, self.embedding_dim, trainable = True , name = 'Decoder_embedding')
        self.decoder_gru = GRU(self.lstm_units, return_state=True, return_sequences=True, name="Decoder_LSTM")


        self.dropout1 = Dropout(0.3,name = 'dropout1')
        self.dropout2 = Dropout(0.3,name = 'dropout2')
        self.dropout3 = Dropout(0.3,name = 'dropout3')

    @tf.function
    def call(self,x,training=None):

        self.input_to_decoder, self.encoder_output, self.state_h = x

        embedded_output = self.decoder_emb(self.input_to_decoder)
        embedded_output = self.dropout1(embedded_output)

        y = [self.state_h,self.encoder_output]
        context_vector, att_weights = self.attention(y)

        concated_decoder_input = tf.concat([tf.expand_dims(context_vector, 1),embedded_output], -1)
        concated_decoder_input = self.dropout2(concated_decoder_input)

        output_gru, hidden_state = self.decoder_gru(concated_decoder_input, initial_state=self.state_h)

        output_gru = tf.reshape(output_gru, (-1, output_gru.shape[2]))
        output_gru = self.dropout3(output_gru)

        output = self.dense(output_gru)

        return output,hidden_state,att_weights,context_vector


class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, lstm_units, attention_units):
        super().__init__()

        self.lstm_units     = lstm_units
        self.vocab_size     = vocab_size
        self.embedding_dim  = embedding_dim
        self.attention_units= attention_units

        self.onestepdecoder = OneStepDecoder(self.vocab_size, self.embedding_dim, self.lstm_units, self.attention_units)

    @tf.function
    def call(self, x,training=None):

        self.input_to_decoder, self.encoder_output, self.decoder_hidden_state = x
        all_outputs = tf.TensorArray(tf.float32,size = self.input_to_decoder.shape[1], name = 'output_arrays' )

        for timestep in tf.range(self.input_to_decoder.shape[1]):

            y = [self.input_to_decoder[:,timestep:timestep+1],self.encoder_output, self.decoder_hidden_state]
            output,hidden_state,att_weights,context_vector = self.onestepdecoder(y)

            self.decoder_hidden_state = hidden_state
            all_outputs = all_outputs.write(timestep,output)

        all_outputs = tf.transpose(all_outputs.stack(),[1,0,2])

        return all_outputs


class Encoder_decoder(tf.keras.Model):
    """
     # Intialize encoder states, Pass the encoder_sequence to the embedding layer
     # Decoder initial states are encoder final states, Initialize it accordingly
     # Pass the decoder sequence,encoder_output,decoder states to Decoder
     # return the decoder output

    """
    def __init__(self, vocab_size, embedding_dim, lstm_units, attention_units, batch_size):
        super().__init__()

        self.vocab_size     = vocab_size
        self.batch_size     = batch_size
        self.lstm_units     = lstm_units
        self.embedding_dim  = embedding_dim
        self.attention_units= attention_units

        self.encoder = Encoder(self.lstm_units)
        self.decoder = Decoder(vocab_size, embedding_dim, lstm_units, attention_units)
        self.dense   = Dense(self.vocab_size, kernel_initializer="glorot_uniform", name = 'enc_dec_dense')


    def call(self,data):

        self.inputs, self.outputs = data[0], data[1]

        self.encoder_hidden = self.encoder.initialize_states(self.batch_size)
        self.encoder_output = self.encoder(self.inputs)

        x = [self.outputs,self.encoder_output,self.encoder_hidden]
        output = self.decoder(x)

        return output

### Optimization adn loss function

In [12]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

### Load model

In [13]:
lstm_units     = 256
embedding_dim  = 300
attention_units= 64
tf.keras.backend.clear_session()
Attention_model = Encoder_decoder(vocab_size,embedding_dim,lstm_units,attention_units,batch_size)
Attention_model.compile(optimizer=tf.keras.optimizers.Adam(0.001),loss=loss_function)
Attention_model.fit(train_dataset, validation_data=validation_dataset, epochs=1, shuffle=True)

64/64 [==============================] - 213s 3s/step - loss: 1.4258 - val_loss: 1.3869


### Load our trained model

In [14]:
Attention_model.load_weights('models/Attention_model03.h5')

### Link for hosting the code in colab

In [ ]:
# from google.colab.output import eval_js
# print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://b7kzrhrzop-496ff2e9c6d22116-5000-colab.googleusercontent.com/


### Prediction, inference and deployment

In [ ]:
# def load_image(img_name):
#   """Loads image in array format"""

#   image = Image.open(img_name)
#   X = np.asarray(image.convert("RGB"))
#   X = np.asarray(X)
#   X = preprocess_input(X)
#   X = resize(X, (256,256,3))
#   X = np.expand_dims(X, axis=0)
#   X = np.asarray(X)

#   return X

# def preprocess(image1_path,image2_path):
#   '''
#     input -- dataframe(df)
#     output -- dataframe(df)
#     process - convert images into 256 X 256, then using CHeXNET model generate tensor(concate two image tensor)
#   '''
#   path = '/content/images/'

#   image_features = []
#   for i in range(len(image1_path)):

#     i1 = load_image(image1_path)
#     i2 = load_image(image2_path)
#     img1_features = model.predict(i1)
#     img2_features = model.predict(i2)
#     img1_features = np.vstack(img1_features).astype(np.float)
#     img2_features = np.vstack(img2_features).astype(np.float)

#     tensor = np.concatenate((img1_features, img2_features), axis=1)

#   return tensor


# def predict_report(image1, image2):
#     '''
#     Input - two image and image path
#     output - return medical report of the images
#     This function takes images and using encoder decoder model
#     return medical report of the images
#     The function predicts the sentence using beam search
#     '''

#     img_tensor     = preprocess(image1, image2)
#     image_features = np.vstack(img_tensor).astype(np.float)

#     result = ''
#     initial_state  = Attention_model.layers[0].initialize_states(1)
#     sequences      = [['<start>', initial_state, 0]]

#     encoder_output       = Attention_model.layers[0](image_features)
#     decoder_hidden_state = initial_state

#     max_len = 75
#     beam_width = 3
#     finished_seq = []

#     for i in range(max_len):
#         new_seq = []
#         all_probable = []

#         for seq,state,score in sequences:

#             cur_vec = np.reshape(word_idx[seq.split(" ")[-1]],(1,1))
#             decoder_hidden_state = state
#             x = [cur_vec, encoder_output, decoder_hidden_state]
#             output,hidden_state,att_weights,context_vector = Attention_model.get_layer('decoder').onestepdecoder(x)
#             output = tf.nn.softmax(output)
#             top_words = np.argsort(output).flatten()[-beam_width:]
#             for index in top_words:

#                 predicted = [seq + ' '+ idx_word[index], hidden_state, score-np.log(np.array(output).flatten()[index])]
#                 all_probable.append(predicted)

#         sequences = sorted(all_probable, key = lambda l: l[2])[:beam_width]

#         count = 0
#         for seq,state,score in sequences:
#             if seq.split(" ")[-1] == '<end>':
#                 score = score/len(seq)
#                 finished_seq.append([seq,state,score])
#                 count+=1
#             else:
#                 new_seq.append([seq,state,score])

#         sequences = new_seq
#         beam_width= beam_width - count
#         if not sequences:
#             break
#         else:
#             continue

#     if len(finished_seq) >0:
#           finished_seq = sorted(finished_seq, reverse=True, key = lambda l: l[2])
#           sequences = finished_seq[-1]
#           return sequences[0][8:-5]
#     else:
#           return new_seq[-1][0]


# from flask import Flask, render_template, request , redirect, url_for, session
# from werkzeug.utils import secure_filename
# import os


# template_folder='templates'
# static_folder='static'

# app = Flask(__name__, template_folder=template_folder,static_folder=static_folder)


# # Ensure static/uploads exists
# UPLOAD_FOLDER = 'static/uploads'

# os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# app.secret_key = 'your_secret_key'  # Needed for session management
# @app.route("/",methods=['GET','POST'])
# def login():
#   USER= 'admin'
#   PASSWORD= 'admin'
#   if request.method=='POST':
#     username= request.form['username']
#     password= request.form['password']

#     if username==USER and password==PASSWORD:
#       session['logged_in'] = True  # Set session variable
#       return render_template("index.html")
#     else:
#       message= "Wrong username or password. Try again!"
#       return render_template("login.html", message=message)

#   else:
#     return render_template("login.html")


# @app.route("/logout")
# def logout():
#     session.pop('logged_in', None)  # Remove session variable
#     return render_template("login.html")



# @app.route('/home')
# def home():
#   # if not session.get('logged_in'):  # Check if user is logged in
#   #       return render_template("login.html")
#   return render_template('index.html')





# @app.route('/predict',methods=['POST'] )
# def predict_caption():


#       front_XRay   = request.files['file_1']
#       lateral_XRay = request.files['file_2']
#       print(front_XRay.filename)
#       print(lateral_XRay.filename)

#       # Save files in static/uploads/
#       front_XRay_path = os.path.join(UPLOAD_FOLDER, secure_filename(front_XRay.filename))
#       lateral_XRay_path = os.path.join(UPLOAD_FOLDER, secure_filename(lateral_XRay.filename))

#       front_XRay.save(front_XRay_path)
#       lateral_XRay.save(lateral_XRay_path)

#       result = predict_report(front_XRay_path,lateral_XRay_path)

#       return render_template('prediction.html', prediction_text=result, front_XRay=secure_filename(front_XRay.filename), lateral_XRay=secure_filename(lateral_XRay.filename))


# if __name__ == '__main__':
#     app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [15]:
!pip install flask-ngrok


In [16]:
!pip install pyngrok


In [17]:
from pyngrok import ngrok
from flask import Flask
ngrok.set_auth_token('2tw2hVGVy5gjl9Tjltx9M4N8j4k_XvtVJELukuPAvfT5hC1Q')
# Specify the port within the config dictionary:
public_url = ngrok.connect(bind_tls=True, addr=5000).public_url
print(public_url)

https://f89f-34-74-133-147.ngrok-free.app


In [ ]:
def load_image(img_name):
  """Loads image in array format"""

  image = Image.open(img_name)
  X = np.asarray(image.convert("RGB"))
  X = np.asarray(X)
  X = preprocess_input(X)
  X = resize(X, (256,256,3))
  X = np.expand_dims(X, axis=0)
  X = np.asarray(X)

  return X

def preprocess(image1_path,image2_path):
  '''
    input -- dataframe(df)
    output -- dataframe(df)
    process - convert images into 256 X 256, then using CHeXNET model generate tensor(concate two image tensor)
  '''
  path = '/content/images/'

  image_features = []
  for i in range(len(image1_path)):

    i1 = load_image(image1_path)
    i2 = load_image(image2_path)
    img1_features = model.predict(i1)
    img2_features = model.predict(i2)
    img1_features = np.vstack(img1_features).astype(np.float)
    img2_features = np.vstack(img2_features).astype(np.float)

    tensor = np.concatenate((img1_features, img2_features), axis=1)

  return tensor


def predict_report(image1, image2):
    '''
    Input - two image and image path
    output - return medical report of the images
    This function takes images and using encoder decoder model
    return medical report of the images
    The function predicts the sentence using beam search
    '''

    img_tensor     = preprocess(image1, image2)
    image_features = np.vstack(img_tensor).astype(np.float)

    result = ''
    initial_state  = Attention_model.layers[0].initialize_states(1)
    sequences      = [['<start>', initial_state, 0]]

    encoder_output       = Attention_model.layers[0](image_features)
    decoder_hidden_state = initial_state

    max_len = 75
    beam_width = 3
    finished_seq = []

    for i in range(max_len):
        new_seq = []
        all_probable = []

        for seq,state,score in sequences:

            cur_vec = np.reshape(word_idx[seq.split(" ")[-1]],(1,1))
            decoder_hidden_state = state
            x = [cur_vec, encoder_output, decoder_hidden_state]
            output,hidden_state,att_weights,context_vector = Attention_model.get_layer('decoder').onestepdecoder(x)
            output = tf.nn.softmax(output)
            top_words = np.argsort(output).flatten()[-beam_width:]
            for index in top_words:

                predicted = [seq + ' '+ idx_word[index], hidden_state, score-np.log(np.array(output).flatten()[index])]
                all_probable.append(predicted)

        sequences = sorted(all_probable, key = lambda l: l[2])[:beam_width]

        count = 0
        for seq,state,score in sequences:
            if seq.split(" ")[-1] == '<end>':
                score = score/len(seq)
                finished_seq.append([seq,state,score])
                count+=1
            else:
                new_seq.append([seq,state,score])

        sequences = new_seq
        beam_width= beam_width - count
        if not sequences:
            break
        else:
            continue

    if len(finished_seq) >0:
          finished_seq = sorted(finished_seq, reverse=True, key = lambda l: l[2])
          sequences = finished_seq[-1]
          return sequences[0][8:-5]
    else:
          return new_seq[-1][0]


from flask import Flask, render_template, request , redirect, url_for, session
from werkzeug.utils import secure_filename
import os
from flask_ngrok import run_with_ngrok



template_folder='templates'
static_folder='static'

app = Flask(__name__, template_folder=template_folder, static_folder=static_folder)

ngrok.set_auth_token('2tw2hVGVy5gjl9Tjltx9M4N8j4k_XvtVJELukuPAvfT5hC1Q')
# Specify the port within the config dictionary:
public_url = ngrok.connect(bind_tls=True, addr=5000).public_url
print(public_url)


# Ensure static/uploads exists
UPLOAD_FOLDER = 'static/uploads'

os.makedirs(UPLOAD_FOLDER, exist_ok=True)

app.secret_key = 'your_secret_key'  # Needed for session management
@app.route("/",methods=['GET','POST'])
def login():
  USER= 'admin'
  PASSWORD= 'admin'
  if request.method=='POST':
    username= request.form['username']
    password= request.form['password']

    if username==USER and password==PASSWORD:
      session['logged_in'] = True  # Set session variable
      return render_template("index.html")
    else:
      message= "Wrong username or password. Try again!"
      return render_template("login.html", message=message)

  else:
    return render_template("login.html")


@app.route("/logout")
def logout():
    session.pop('logged_in', None)  # Remove session variable
    return render_template("login.html")



@app.route('/home')
def home():
  # if not session.get('logged_in'):  # Check if user is logged in
  #       return render_template("login.html")
  return render_template('index.html')





@app.route('/predict',methods=['POST'] )
def predict_caption():


      front_XRay   = request.files['file_1']
      lateral_XRay = request.files['file_2']
      print(front_XRay.filename)
      print(lateral_XRay.filename)

      # Save files in static/uploads/
      front_XRay_path = os.path.join(UPLOAD_FOLDER, secure_filename(front_XRay.filename))
      lateral_XRay_path = os.path.join(UPLOAD_FOLDER, secure_filename(lateral_XRay.filename))

      front_XRay.save(front_XRay_path)
      lateral_XRay.save(lateral_XRay_path)

      result = predict_report(front_XRay_path,lateral_XRay_path)

      return render_template('prediction.html', prediction_text=result, front_XRay=secure_filename(front_XRay.filename), lateral_XRay=secure_filename(lateral_XRay.filename))


if __name__ == '__main__':
    app.run(port=5000)


https://5678-34-74-133-147.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:07:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:07:57] "GET /static/styles/mystyle.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:07:58] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:08:16] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:08:17] "GET /static/js/myjs.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:08:17] "GET /static/styles/index.css HTTP/1.1" 200 -


CXR1_1_IM-0001-4001.png
CXR1_1_IM-0001-3001.png
1/1 [==============================] - 0s 423ms/step


INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:09:14] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:09:15] "GET /static/uploads/CXR1_1_IM-0001-3001.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:09:15] "GET /static/uploads/CXR1_1_IM-0001-4001.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:09:15] "GET /static/styles/predict.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:12:06] "GET /home HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:12:07] "GET /static/styles/index.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:12:07] "GET /static/js/myjs.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:12:28] "GET /home HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:12:29] "GET /static/styles/index.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:12:29] "GET /static/js/myjs.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:21:15] "GET /logout

test0.png
test00.png
1/1 [==============================] - 0s 262ms/step


INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:29:43] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:29:44] "GET /static/uploads/test0.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:29:44] "GET /static/styles/predict.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:29:44] "GET /static/uploads/test00.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:30:08] "GET /home HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:30:09] "GET /static/styles/index.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:30:09] "GET /static/js/myjs.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:30:11] "GET /logout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:30:11] "GET /static/styles/mystyle.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:45:01] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:45:02] "GET /static/styles/index.css HTTP/1.1" 304 -
INFO:

test0.png
test00.png
1/1 [==============================] - 0s 288ms/step


INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:45:41] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:45:41] "GET /static/uploads/test0.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:45:41] "GET /static/styles/predict.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:45:41] "GET /static/uploads/test00.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:45:48] "GET /home HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:45:49] "GET /static/js/myjs.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:45:49] "GET /static/styles/index.css HTTP/1.1" 304 -


test11.png
test1.png
1/1 [==============================] - 0s 280ms/step


INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:46:26] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:46:26] "GET /static/styles/predict.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:46:26] "GET /static/uploads/test11.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 06:46:26] "GET /static/uploads/test1.png HTTP/1.1" 200 -
